## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [ ]:
path_img = '/content/drive/MyDrive/kt_mini_pj_4/cool_roof_images.zip'
path_txt = '/content/drive/MyDrive/kt_mini_pj_4/cool_roof_yolo_labels.zip'

In [ ]:
path_img[36:-4]

'cool_roof_images'

In [ ]:
import zipfile, os, glob, shutil
from tqdm import tqdm

In [ ]:
def dataset_extract(file_name) :
    with zipfile.ZipFile(file_name, 'r') as zip_ref :
        file_list = zip_ref.namelist()

        if os.path.exists(f'./{file_name[36:-4]}/') :
            print('데이터셋 폴더가 이미 존재합니다.')
            return

        else :
            for f in tqdm(file_list, desc='Extracting', unit='files') :
                zip_ref.extract(member=f, path=f'./{file_name[36:-4]}/')

In [ ]:
dataset_extract(path_img)
dataset_extract(path_txt)

Extracting: 100%|██████████| 204/204 [00:00<00:00, 1223.53files/s]


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [ ]:
def make_folder() :
    if os.path.exists('./cool_roof_dataset/') :
        print('이미 만들어져 있습니다. 종료하겠음.')
        return

    os.makedirs('./cool_roof_dataset/')

    os.makedirs('./cool_roof_dataset/train/')
    os.makedirs('./cool_roof_dataset/train/images')
    os.makedirs('./cool_roof_dataset/train/labels')

    os.makedirs('./cool_roof_dataset/valid/')
    os.makedirs('./cool_roof_dataset/valid/images')
    os.makedirs('./cool_roof_dataset/valid/labels')

In [ ]:
make_folder()

In [ ]:
print( os.listdir('./cool_roof_dataset/') )
print( os.listdir('./cool_roof_dataset/train/') )
print( os.listdir('./cool_roof_dataset/valid/') )

['train', 'valid']
['images', 'labels']
['images', 'labels']


### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
list_img = sorted(glob.glob('./cool_roof_images/*.jpg') )

In [29]:
list_img[0][19:]
#list_txt[0][39:]

'roof (1).jpg'

In [ ]:
list_txt = sorted(glob.glob('./cool_roof_yolo_labels/obj_train_data/*.txt'))

In [ ]:
def moving_data() :
    try :
        img_tr_list, img_val_list, txt_tr_list, txt_val_list =\
            train_test_split( list_img,
                              list_txt,
                              random_state=2024, test_size=0.2
                              )

        for img in img_tr_list :
            shutil.move(src=img, dst=f'./cool_roof_dataset/train/images/{img[19:]}') 
            # shutil = 파일이나 디렉토리 이동, 복사할때 사용되는 모듈
            # src = 이동하려는 파일 이름이나 디렉토리 경로, dst = 저장지점
            # img의 이름 19번째 자리 부터 씀
        for txt in txt_tr_list :
            shutil.move(src=txt, dst=f'./cool_roof_dataset/train/labels/{txt[39:]}')

        for img in img_val_list :
            shutil.move(src=img, dst=f'./cool_roof_dataset/valid/images/{img[19:]}')
        for txt in txt_val_list :
            shutil.move(src=txt, dst=f'./cool_roof_dataset/valid/labels/{txt[39:]}')

    except :
        print('이미 실행된 함수이다.')

In [ ]:
moving_data()

### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [ ]:
os.listdir('./cool_roof_yolo_labels')

['obj.names', 'train.txt', 'obj.data', 'obj_train_data']

In [ ]:
with open('./cool_roof_yolo_labels/obj.names') as f :
    strings = f.readlines()
    # print(strings)
    c_roof, g_roof = strings[0].strip(), strings[1].strip()

roof = {0:c_roof, 1:g_roof}

In [ ]:
roof

{0: 'cool roof', 1: 'generic roof'}

In [ ]:
yaml_dict = {'train':'./train/images',
             'val':'./valid/images',
             'names':roof
             }

yaml_dict

{'train': './train/images',
 'val': './valid/images',
 'names': {0: 'cool roof', 1: 'generic roof'}}

In [ ]:
import yaml

In [ ]:
# yaml 파일 생성

with open('./cool_roof_dataset/roof_info.yaml', 'w') as f :
    yaml.dump(yaml_dict, f)

### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO(model='yolov8n.pt', task='detect')

100%|██████████| 6.23M/6.23M [00:00<00:00, 74.6MB/s]


In [ ]:
model.train(data='/content/cool_roof_dataset/roof_info.yaml',
            epochs=10,
            patience=5
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/cool_roof_dataset/roof_info.yaml, epochs=10, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

100%|██████████| 755k/755k [00:00<00:00, 14.6MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/cool_roof_dataset/train/labels... 160 images, 0 backgrounds, 0 corrupt: 100%|██████████| 160/160 [00:00<00:00, 431.14it/s]

train: New cache created: /content/cool_roof_dataset/train/labels.cache


...

                   all         40        304       0.72      0.664      0.722        0.5



10 epochs completed in 0.423 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:14<00:00,  7.23s/it]


                   all         40        304       0.72      0.668      0.722      0.499
             cool roof         40         38      0.738      0.444      0.606       0.44
          generic roof         40        266      0.701      0.892      0.838      0.557
Speed: 7.2ms preprocess, 287.9ms inference, 0.0ms loss, 10.9ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7afe3b1ace20>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [ ]:
model.predict(source='/content/cool_roof_dataset/valid/images',
              save=True,
              line_width=2)


image 1/40 /content/cool_roof_dataset/valid/images/roof (1).jpg: 640x640 9 generic roofs, 195.4ms
image 2/40 /content/cool_roof_dataset/valid/images/roof (10).jpg: 640x512 1 cool roof, 7 generic roofs, 184.3ms
image 3/40 /content/cool_roof_dataset/valid/images/roof (112).jpg: 640x640 4 generic roofs, 197.3ms
image 4/40 /content/cool_roof_dataset/valid/images/roof (116).jpg: 480x640 4 generic roofs, 156.7ms
image 5/40 /content/cool_roof_dataset/valid/images/roof (118).jpg: 640x576 1 cool roof, 12 generic roofs, 183.8ms
image 6/40 /content/cool_roof_dataset/valid/images/roof (126).jpg: 640x576 7 generic roofs, 201.0ms
image 7/40 /content/cool_roof_dataset/valid/images/roof (129).jpg: 640x608 9 generic roofs, 194.3ms
image 8/40 /content/cool_roof_dataset/valid/images/roof (13).jpg: 544x640 2 cool roofs, 11 generic roofs, 191.8ms
image 9/40 /content/cool_roof_dataset/valid/images/roof (132).jpg: 352x640 5 generic roofs, 122.7ms
image 10/40 /content/cool_roof_dataset/valid/images/roof (133

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'cool roof', 1: 'generic roof'}
 obb: None
 orig_img: array([[[ 57,  46,  32],
         [ 56,  44,  32],
         [ 57,  46,  32],
         ...,
         [ 26,  17,   8],
         [ 27,  17,   7],
         [ 27,  18,   5]],
 
        [[ 59,  46,  32],
         [ 54,  42,  30],
         [ 58,  45,  31],
         ...,
         [ 26,  17,   8],
         [ 27,  17,   7],
         [ 27,  18,   5]],
 
        [[ 59,  46,  32],
         [ 58,  45,  31],
         [ 58,  47,  33],
         ...,
         [ 26,  17,   8],
         [ 27,  18,   5],
         [ 28,  19,   6]],
 
        ...,
 
        [[118, 132, 126],
         [124, 136, 130],
         [129, 144, 136],
         ...,
         [114, 130, 123],
         [115, 130, 126],
         [116, 131, 127]],
 
        [[114, 129, 121],
         [116, 130, 124],
         [121, 136, 128],
         .

In [ ]:
np.random.randint(1, 39)

NameError: name 'np' is not defined